In [131]:
import numpy as np
import time

def generate_distance_matrix(n):
    """
    Generates n random points in R^3 and computes their distance matrix.

    Parameters:
        n (int): Number of points.

    Returns:
        numpy.ndarray: An n x n distance matrix.
    """
    # Generate n random points in R^3
    points = np.random.rand(n, 3)

    # Compute the distance matrix
    distance_matrix = np.sqrt(
        np.sum((points[:, np.newaxis, :] - points[np.newaxis, :, :])**2, axis=2)
    )
    return points, distance_matrix
# Example Usage
if __name__ == "__main__":
    n = 250
    points, D = generate_distance_matrix(n)
    D[0, 1] = 1000  # Corrupt the distance between the first two points
    D[1, 0] = 1000

    corruptions = find_corruptions(D)
    corruptions = find_corruptions_naive(D)

Returing best location: None
Corruption test took 0.02 seconds
Corrupted triplets (naive): [(0, 1)]
Time taken (naive): 3.951404094696045


In [58]:
import numpy as np

def find_corruptions(D, max_deviation=1.0):
    start_time = time.time()
    n = len(D)
    max_diff = 0
    best_location = None
    
    for i in range(n):
        for j in range(i+1, n):
            diff = abs(D[i,j] - D[j,i])
            
            if diff > max_deviation and diff > max_diff:
                max_diff = diff
                best_location = (i, j)
    print(f"Returing best location: {best_location}")
    print(f"Corruption test took {time.time() - start_time:.2f} seconds")

    return best_location, max_diff

In [40]:
import numpy as np

def find_corruptions_naive(distance_matrix):
    """
    Naive method to find corruptions in a distance matrix by checking the triangle inequality.

    Parameters:
        distance_matrix (numpy.ndarray): An n x n symmetric distance matrix.

    Returns:
        list: A list of tuples (i, j) where the distance matrix violates the triangle inequality.
    """
    start_time = time.time()
    n = distance_matrix.shape[0]
    corruptions = []

    # Check all triplets (i, j, k) for violation of the triangle inequality
    for i in range(n):
        for j in range(i + 1, n):  # Only need to check upper triangle
            for k in range(n):
                if k != i and k != j:  # Don't check the pair (i, j) itself
                    if distance_matrix[i, j] > distance_matrix[i, k] + distance_matrix[k, j]:
                        corruptions.append((i, j))
                        break  # If we find a corruption, no need to check further for this pair

    print(f"Corrupted triplets (naive): {corruptions}")
    print(f"Time taken (naive): {time.time() - start_time}")
    return corruptions


In [132]:
def get_L(n):
    s = np.zeros(n, dtype=int)
    s[0] = 1
    e = np.ones(n, dtype=int)
    outer_product = s[:, None] @ e[None, :]  # s[:, None] makes s a column vector, e[None, :] makes e a row vector
    
    L = np.eye(n) - outer_product
    return L

def check_valid_dist(D):
    """
    Given a distnace matrix D, check if its gram matrix is PSD
    F = -L^T D L
    """
    n = len(D)
    L = get_L(n)
    F = -L.T @ D @ L
    return "valid" if np.all(np.linalg.eigvals(F) >= 0) else "invalid"

In [129]:
n = 250
points, D = generate_distance_matrix(n)
corruption = 0.5
D[0, 1] = corruption  # Corrupt the distance between the first two points
D[1, 0] = corruption
check_PSD(D)


'valid'

In [130]:
import torch

# Define the size of the matrix and the function to get L
n = len(D)  # Assuming D is predefined
L = torch.tensor(get_L(n), dtype=torch.float64, requires_grad=True)  # Assuming get_L is a function that gives L
D = torch.tensor(D, dtype=torch.float64, requires_grad=True)  # D is a predefined matrix

# Compute F = -L^T D L
F = -L.T @ D @ L

# Compute the smallest eigenvalue
eigenvalues = torch.linalg.eigvalsh(F)  # Returns eigenvalues in sorted order (ascending)
lambda_min = eigenvalues.min()  # Take the smallest eigenvalue
print(eigenvalues)
# Compute gradients with respect to D
lambda_min.backward()  # Backpropagate to compute gradients w.r.t. D

# Get the gradients
gradients = D.grad

# Output the gradients (gradient with respect to each entry in D)
print(gradients)

# Identify the most problematic entry (the one with the highest absolute gradient)
problematic_entry = torch.argmax(torch.abs(gradients))

# Print the most problematic entry
print(f"The most problematic entry in D is: {problematic_entry}")


tensor([0.0000e+00, 2.7577e-02, 2.8040e-02, 2.8998e-02, 3.1536e-02, 3.3227e-02,
        3.4576e-02, 3.5614e-02, 3.5973e-02, 3.6640e-02, 3.6841e-02, 3.7054e-02,
        3.7796e-02, 3.7954e-02, 3.9502e-02, 3.9762e-02, 4.0470e-02, 4.1851e-02,
        4.2415e-02, 4.2461e-02, 4.2992e-02, 4.4015e-02, 4.4355e-02, 4.4949e-02,
        4.5456e-02, 4.5615e-02, 4.5893e-02, 4.7121e-02, 4.7366e-02, 4.7481e-02,
        4.7570e-02, 4.7774e-02, 4.8756e-02, 4.8762e-02, 4.8835e-02, 4.9867e-02,
        5.0115e-02, 5.0795e-02, 5.0866e-02, 5.1432e-02, 5.1829e-02, 5.2066e-02,
        5.2529e-02, 5.3197e-02, 5.3399e-02, 5.3507e-02, 5.4421e-02, 5.4881e-02,
        5.5447e-02, 5.5931e-02, 5.6529e-02, 5.6732e-02, 5.6912e-02, 5.7554e-02,
        5.8377e-02, 5.9145e-02, 5.9635e-02, 6.0021e-02, 6.0399e-02, 6.0806e-02,
        6.1373e-02, 6.2404e-02, 6.2724e-02, 6.3316e-02, 6.3846e-02, 6.3930e-02,
        6.5000e-02, 6.5904e-02, 6.6384e-02, 6.6610e-02, 6.6815e-02, 6.7263e-02,
        6.7734e-02, 6.7908e-02, 6.9022e-